In [1]:
#| default_exp data

In [2]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [3]:
#| export
from fastcore.utils import *
import pandas as pd
from pandas import DataFrame
import numpy as np
import pyreadstat
import pyspssio
from pathlib import Path
from typing import Dict, List, Tuple, Optional

Define a function for reading an SPSS file, converting the metadata into a dataframe, and then saving the data and metadata to parquet files.

In [4]:
#| export
def reformat_metadata(m1: pyreadstat.metadata_container, # metadata from pyreadstat
                      m2: Dict[str, Dict], # metadata from pyspssio
                      ) -> DataFrame:
      "Combine metadata from pyreadstat and pyspssio and convert into a pandas DataFrame."
      meta={"Label": m1.column_names_to_labels,
            "Field Type": m1.original_variable_types, # Pyreadstat version
            "Field Width": m1.variable_display_width,
            "Decimals": {k: v[2] for k, v in m2['var_formats_tuple'].items()},
            "Variable Type": m1.variable_measure,
            "Field Values": m1.variable_value_labels}
      return DataFrame(data={k: meta[k] for k in meta.keys()}).T

In [5]:
#| hide
file = "../data/G227_Q.sav"
_, meta = pyreadstat.read_sav(file)
_, meta2 = pyspssio.read_sav(file)
meta = reformat_metadata(meta, meta2)

test_eq(type(meta), DataFrame)
test_eq(meta.index, ['Label', 'Field Type', 'Field Width', 'Decimals', 'Variable Type', 'Field Values'])

Create a function to read an SPSS file.
Read with two different packages, and compare results; raise warning where differences occur.
Combine and filter the metadata from the packages and output it as a DataFrame.

In [24]:
#| export
# TODO: write functions to compare df and meta between the packages
def read_sav(file: str, # Path to SPSS file
             index: Optional[str] = None, # column to set as index
            ) -> Tuple[DataFrame, DataFrame]: # Output df and meta as dataframes
      "Wrapper around `pyreadstat.read_sav()` with nicer metadata output."
      _, meta = pyreadstat.read_sav(file)
      df, meta2 = pyspssio.read_sav(file)
      if index: df = df.set_index(index).sort_index()
      meta = reformat_metadata(meta, meta2)
      return df, meta

Check for any differences in how metadata is read between `pyreadstat` and `pyspssio`.

In [7]:
df, meta = pyreadstat.read_sav(file)
df2, meta2 = pyspssio.read_sav(file)

In [8]:
# Variable names
test_eq(meta.column_names, meta2['var_names'])

# Variable labels
test_eq(meta.column_names_to_labels, meta2['var_labels'])

# Field type
# test_eq(meta.original_variable_types, meta2["var_formats"]) # TODO: resolve differences

# Field width
test_eq(meta.variable_display_width, meta2['var_column_widths'])

# Decimals


# Variable type
test_eq(meta.variable_measure, meta2['var_measure_levels'])

# Field values
# test_eq(meta.variable_value_labels, meta2['var_value_labels']) # TODO: how to determine which variables have been dropped?

There are difference in types, which appears to simply be because the packages handle integers differently.

In [9]:
DataFrame(meta.original_variable_types, index=[0]).T.compare(DataFrame(meta2["var_formats"], index=[0]).T)

0      
                    self other
G227_PCBY1          F4.0    F4
G227_PCBY2          F4.0    F4
G227_PCBY3          F4.0    F4
G227_PCBY4          F4.0    F4
G227_BU1A           F3.0    F3
...                  ...   ...
G227_DQ_CHOC_PD     F8.0    F8
G227_DQ_ICECHOC     F8.0    F8
G227_DQ_ICECHOC_PD  F8.0    F8
G227_DQ_EATCHOC     F8.0    F8
G227_DQ_CAFFTAB     F8.0    F8

[154 rows x 2 columns]

In [10]:
m = DataFrame(meta.original_variable_types, index=['Field Type']).T
m['Type'] = m['Field Type'].map(lambda s: s[0])

In [11]:
m['Type'].unique()

array(['F', 'D', 'A', 'T'], dtype=object)

In [12]:
m.loc[m['Type'] == 'A']

,Field Type,Type
G227_DWEL_OTH,A73,A
G227_LIV8_OTH,A14,A
G227_BNF9_OTH,A140,A
G227_HINS4_OTH,A67,A
G227_ED34_OTH,A66,A
...,...,...
G227_HEAR_DIZ_T4A,A31,A
G227_HEAR_SYM7A,A39,A
G227_HEAR_DIZ11A,A35,A
G227_TATT_ADV_DES,A192,A


Then define a function for saving SPSS files

In [13]:
#| export
def output_metadata(metadata: pd.DataFrame):
    "Convert metadata from a DataFrame to a nested dictionary to be compatible with pyspssio"
    RENAME_COLUMNS = {
        "Label": "var_labels",
        "Field Type": "var_formats",
        "Field Width": "var_column_widths",
        # "Decimals": , # could write a fn with regex to determine based on var format / field type
        "Variable Type": "var_measure_levels",
        "Field Values": "var_value_labels"
    }
    metadata = (metadata
                .rename(index=RENAME_COLUMNS)
                .T
                .to_dict())
    # Remove instances where values are NaN for compatibility when saving with pyspssio
    metadata = {key: {k: v for k, v in value.items() if not pd.isnull(v)} for key, value in metadata.items()}
    
    return metadata

In [23]:
#| export
def write_sav(output: str, # path to save file
         df: DataFrame, # raw data (reset index if it's set to "ID")
         meta: DataFrame = None # metadata in DataFrame format
         ) -> None:
    "Wrapper around pyspssio write_sav to handle metadata conversion."
    if df.index.name == "ID": df.reset_index(inplace=True)
    if meta is not None: meta = output_metadata(meta)
    pyspssio.write_sav(output, df, meta)

## Parquet Data

Next, write a function to save data as a parquet file for a more efficient storage format.

In [15]:
#| hide
def sav_to_parquet(df: DataFrame, #
                   meta: DataFrame, # 
                   filename: str, # Basename for saving files (ie. for G208_Q.sav, filename="G208_Q")
                   dir: str # Directory to save output
                   ) -> None:
      "Save data and metadata as parquet files."
      # Convert metadata to all string types so it behaves nicely when saving as a parquet file
      meta = meta.astype(str)
      df.to_parquet(Path(dir) / f"{filename}_df.parquet")
      meta.to_parquet(Path(dir) / f"{filename}_meta.parquet")

In [16]:
df, meta = read_sav(file)
sav_to_parquet(df, meta, 'G227_Q', '../data/')

Verify that there is no loss or corruption of data in the conversion process.

In [17]:
df_pq = pd.read_parquet("../data/G227_Q_df.parquet")
test_eq(df, df_pq)

In [18]:
meta_pq = pd.read_parquet("../data/G227_Q_meta.parquet")
test_eq(meta.astype(str), meta_pq)

## Dataset

Create dataset class that holds the raw data and metadata.

In [19]:
#| export
class Dataset:
    "A class which contains both the data and metadata for a given data file."
    def __init__(self,
                 df: DataFrame, # the actual raw data
                 meta: DataFrame): # the metadata, including variable labels, value labels, and types for each variable
        self.df, self.meta = df, meta

In [25]:
#| hide
import nbdev; nbdev.nbdev_export()